# Good Note book for reference
#    1.    https://www.kaggle.com/mistag/data-preprocessing-with-gwpy 
# Special thanks to Darek, he made data conversion from numpy to Qtransformed png file "https://www.kaggle.com/thedrcat/g2net-train-images-with-gpwy-sample/"
# I thought I could levrage that for now, to create base line 

# Adding Note book output to dataset
# https://www.kaggle.com/getting-started/168312

In [ ]:
import pandas as pd
import numpy as np
from scipy import fft
from scipy.signal import spectrogram
from scipy import signal
from os import path

In [ ]:
try:
    from gwpy.timeseries import TimeSeries
    from gwpy.plot import Plot
    
except:
    !python -m pip install gwpy
    !pip install astropy==4.2.1
    from gwpy.timeseries import TimeSeries
    from gwpy.plot import Plot

In [ ]:
from pandarallel import pandarallel
pandarallel.initialize()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly 
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from PIL import Image

In [ ]:
try:
    import efficientnet.keras as efn
except:
    
    !pip install -U efficientnet
    import efficientnet.keras as efn


import tensorflow as tf 
import glob
from sklearn.model_selection import train_test_split

In [ ]:
sns.set_theme("talk")
sns.set_style("white")

In [ ]:
train_file ="../input/g2net-gravitational-wave-detection/training_labels.csv"
submission_file ="../input/g2net-gravitational-wave-detection/sample_submission.csv"

In [ ]:
master_df = pd.read_csv( train_file)
master_df.head()

# Constructing complete path, it will be very helpfull for further analysis

In [ ]:
master_df["path"]= master_df["id"].parallel_apply( lambda x : "../input/g2net-gravitational-wave-detection/train/" +x[0] +"/"+x[1] +"/"+x[2] +"/" + x+".npy")

In [ ]:
master_df.head()

# Check number of Singals present in dataframe

In [ ]:
print (" Number of Signal samples present ={}".format( master_df.shape[0] ) )

# Check path constructed correct or not 

In [ ]:
file_exist = False

for i in range( 0, 20 ):
    file = master_df["path"].iloc[ np.random.randint( master_df.shape[0] ) ]
    if not path.exists( file ): 
        print ( "File does not exist, constructed path is wrong for file = {}".format( file ) )
        file_exist = True
if not file_exist : print ("All 20 file are present ")

# Visualize distribution of traget

In [ ]:
fig = plt.figure( figsize = (5,5), dpi = 90 )
sns.countplot( master_df["target"] )
plt.show()

# Good part is data is not imbalanced

# Let us see how datais stored
1. Format
2. Number of channels

In [ ]:
sample_file = master_df["path"].iloc[0]
target= master_df["target"].iloc[0]
data = np.load( sample_file  )
data.shape

In [ ]:
## Data is stored in 2 dimension
for i in range( 0, 3 ):
    file,target  = master_df[["path","target"]].iloc[np.random.randint(master_df.shape[0] ) ].values
    data = np.load( file )
    fig = go.Figure()
    fig.add_trace( go.Line( y = data[0], name ="LIGO Hanford" ) )
    fig.add_trace( go.Line( y = data[1], name ="LIGO Livingston" ) )
    fig.add_trace( go.Line( y = data[2], name ="LIGO Virgo" ) )
    fig.update_layout(title ="Gravitation wave in Time domain from 3 Detectors and target={}".format( target ) )
    fig.show()

# Measured amplitude is very small range lets visualize distrbution of values using histogram 

In [ ]:
file_name, target= master_df[["path","target"]].iloc[np.random.randint( master_df.shape[0])].values
data = np.load( file_name)
    
fig = go.Figure()
fig.add_trace( go.Histogram( x = data[0], autobinx= True, name = "LIGO Hanford target={}".format( target ) ))
fig.add_trace( go.Histogram( x = data[0], autobinx= True,name = "LIGO Livingston target={}".format( target ) ))
fig.add_trace( go.Histogram( x = data[0], autobinx= True,name = "LIGO Virgo target={}".format( target ) ))
fig.show()

# let usscale values by 10^-20 and visualize the distribution 

In [ ]:
file_name, target= master_df[["path","target"]].iloc[np.random.randint( master_df.shape[0])].values
data = np.load( file_name)
data = np.multiply( data, 10**20 )    
fig = go.Figure()
fig.add_trace( go.Histogram( x = data[0], autobinx= True, name = "LIGO Hanford target={}".format( target ) ))
fig.add_trace( go.Histogram( x = data[0], autobinx= True,name = "LIGO Livingston target={}".format( target ) ))
fig.add_trace( go.Histogram( x = data[0], autobinx= True,name = "LIGO Virgo target={}".format( target ) ))
fig.show()

## Time domain will provide only little info.
## Frequency domain should give more detils on singal strenght and other frequency componenets.
## Convert amplitude to db using formula 20log(data)

In [ ]:
fig = make_subplots( rows = 1, cols = 2 )
for i in range( 0, 2):
    file_name, target= master_df[["path","target"]].iloc[np.random.randint( master_df.shape[0])].values
    data = np.load( file_name)
    data = np.multiply( data, 10**20 )    
    fig.add_trace( go.Line( x = fft.rfftfreq( 2* 2048, (1/ 4096)) , y = 20*np.log(np.abs( fft.rfft( data[0]) )),name ="LIGO Hanford target={}".format( target )), row = 1, col = i+1 )
    fig.add_trace( go.Line( x = fft.rfftfreq( 2* 2048, (1/ 4096)) , y = 20*np.log(np.abs( fft.rfft( data[1]) )),name ="LIGO Livingston target={}".format( target )), row = 1, col = i+1 )
    fig.add_trace( go.Line( x = fft.rfftfreq( 2* 2048, (1/ 4096)) , y = 20*np.log(np.abs( fft.rfft( data[2]) )),name ="LIGO Virgo target={}".format( target ) ) , row = 1, col = i+1)

fig.update_layout( title ="Gravitational wave inFreq Domain ")
fig.update_xaxes( title ="Freq in Hz")
fig.update_yaxes( title ="Signal Power in dB")
fig.show()


## we can't analyze time domain singal or Frequency domain signal using Machine learning model.
## Let us convert singal to SFT ( Short fourier Transform ) and then to Constant Q Transform

In [ ]:
file_name, target= master_df[["path","target"]].iloc[np.random.randint( master_df.shape[0])].values
data = np.load( file_name)

In [ ]:
f,t,mag = spectrogram( np.multiply( data[0], 10**20) , fs = 2048, window = "hamming", noverlap= 100, mode ="magnitude")
fig = go.Figure()
fig.add_trace( go.Heatmap( x= t, y = f, z = np.abs(mag) ))
fig.show()

# I came across one note book very informative, "https://www.kaggle.com/mistag/data-preprocessing-with-gwpy"
# Talks about converting giventime series data to Gwavitational wave using gwpy library.
# Taken implementation and code from same note book.


In [ ]:

## Fucntion converts data to GW wave format
def read_file(fname):
    data = np.load(fname)
    d1 = TimeSeries(data[0,:], sample_rate=2048)
    d2 = TimeSeries(data[1,:], sample_rate=2048)
    d3 = TimeSeries(data[2,:], sample_rate=2048)
    return d1, d2, d3

def plot_time_data(d1, d2, d3):
    plot = Plot(d1, d2, d3, separate=True, sharex=True, figsize=[12, 8])
    ax = plot.gca()
    #ax.set_title(["1","2","3"])
    ax.set_xlim(0,2)
    ax.set_xlabel('Time [s]')
    plt.tight_layout()
    plot.show( )

In [ ]:
d1, d2, d3 = read_file('../input/g2net-gravitational-wave-detection/train/0/0/0/000a5b6e5c.npy')
plot_time_data(d1, d2, d3)


In [ ]:
#applying window function, i.e creating Band pass filter 

window = signal.tukey( 4096 )
plt.plot( window)



# Data after applying Band pass filter

In [ ]:
d1, d2, d3 = d1*window, d2*window, d3*window
plot_time_data(d1, d2, d3)

# Frequency domaain data after passing through High Pass filter


In [ ]:
fig2b = d1.highpass(15).asd(fftlength=2).plot(figsize=[12, 6])
plt.xlim(10,1024)
plt.ylim(1e-25, 1e-20);


# Spectral whitening and bandpass filtering

# This is super simple with GWpy:


In [ ]:
white_data = d1.whiten(window=("tukey",0.2)) # whiten-function has a built-in window function
bp_data = white_data.bandpass(35, 350) # frequency range 35-350Hz
fig3 = bp_data.plot(figsize=[12, 6])
plt.xlim(0, 2)
ax = plt.gca()
ax.set_title('Whitened and bandpassed')
ax.set_xlabel('Time [s]')
ax.set_ylabel( "amplitude ")


#  Now, we have a preprocessed data that is ready for further analysis. First, let's define a function that combines all the steps above and outputs preprocessed data:


In [ ]:

#lf & hf are cut of freq for Band pass filter
def preprocess(d1, d2, d3, bandpass=False, lf=20, hf=350):
    white_d1 = d1.whiten(window=("tukey",0.2))
    white_d2 = d2.whiten(window=("tukey",0.2))
    white_d3 = d3.whiten(window=("tukey",0.2))
    if bandpass: # bandpass filter
        bp_d1 = white_d1.bandpass(lf, hf) 
        bp_d2 = white_d2.bandpass(lf, hf)
        bp_d3 = white_d3.bandpass(lf, hf)
        return bp_d1, bp_d2, bp_d3
    else: # only whiten
        return white_d1, white_d2, white_d3


# Q-Transform

# The Q-Transform is related to the Fourier transform, and very closely related to a wavelet transform. The spectrogram is a possible candidate as input for a CNN model.


In [ ]:
r1, r2, r3 = read_file('../input/g2net-gravitational-wave-detection/train/0/0/0/000a5b6e5c.npy') # this signal has target=1
p1, p2, p3 = preprocess(r1, r2, r3)
hq = p2.q_transform(qrange=(16,32), frange=(30,400), logf=True, whiten=False)
fig4 = hq.plot(figsize=[12, 10])
ax = fig4.gca()
fig4.colorbar(label="Normalised energy")
ax.grid(False)
ax.set_yscale('log')
ax.set_xlabel('Time [s]');

In [ ]:
print ( "Shape of Transoformationof single detector is ={}".format(hq.shape) )

# Combine three channels into one RGB image
# Since we have 3 detectors, we can combine the Q-Transforms as RGB channels into one color image. Let's make a function for that:

In [ ]:
Q_RANGE = (16,32)
F_RANGE = (30,400)

def create_rgb(fname):
    r1, r2, r3 = read_file(fname)
    p1, p2, p3 = preprocess(r1, r2, r3)
    hq1 = p1.q_transform(qrange=Q_RANGE, frange=F_RANGE, logf=True, whiten=False)
    hq2 = p2.q_transform(qrange=Q_RANGE, frange=F_RANGE, logf=True, whiten=False)
    hq3 = p3.q_transform(qrange=Q_RANGE, frange=F_RANGE, logf=True, whiten=False)
    img = np.zeros([hq1.shape[0], hq1.shape[1], 3], dtype=np.uint8)
    scaler = MinMaxScaler()
    img[:,:,0] = 255*scaler.fit_transform(hq1)
    img[:,:,1] = 255*scaler.fit_transform(hq2)
    img[:,:,2] = 255*scaler.fit_transform(hq3)
    return Image.fromarray(img).rotate(90, expand=1).resize((760,760))


In [ ]:
create_rgb('../input/g2net-gravitational-wave-detection/train/0/0/0/000a5b6e5c.npy')

# Train data might not give good View, let us vissualize test data

In [ ]:
create_rgb( "../input/g2net-gravitational-wave-detection/test/1/0/b/10b041376b.npy")

# Did not good, but decent amount of EDA lets start with building model and data pipeline

In [ ]:


## Fucntion converts data to GW wave format
def read_file(fname):
    data = np.load(fname)
    d1 = TimeSeries(data[0,:], sample_rate=2048)
    d2 = TimeSeries(data[1,:], sample_rate=2048)
    d3 = TimeSeries(data[2,:], sample_rate=2048)
    return d1, d2, d3



def convert_data_3channel_qtran( file_name ,Q_range_low = 16, q_range_high = 32, f_range_ll = 10, f_range_high = 400  ):
    r1, r2, r3 = read_file(file_name)
    p1, p2, p3 = preprocess(r1, r2, r3)
    hq1 = p1.q_transform(qrange=(Q_range_low, q_range_high), frange=( f_range_ll, f_range_high), logf=True, whiten=False)
    hq2 = p2.q_transform(qrange=(Q_range_low, q_range_high), frange=( f_range_ll, f_range_high), logf=True, whiten=False)
    hq3 = p3.q_transform(qrange=(Q_range_low, q_range_high), frange=( f_range_ll, f_range_high), logf=True, whiten=False)
    #img = np.zeros([hq1.shape[0], hq1.shape[1], 3], dtype=np.float32)
    scaler = MinMaxScaler()
    hq1 = 255*scaler.fit_transform(hq1)
    hq2 = 255*scaler.fit_transform(hq2)
    hq3=  255*scaler.fit_transform(hq3)
    img = np.dstack( ( hq1, hq2, hq3 ) )
    return img #Image.fromarray(img).rotate(90, expand=1).resize((256,256))

if False:
    
    for each_file in glob.glob( "../input/g2net-gravitational-wave-detection/train/*/*/*/*.npy")[:10]:

        convert_data_3channel_qtran(each_file).save("./"+each_file.split("/")[-1].replace(".npy",".png"))

        print( each_file)
    
    

In [ ]:
CFG= {"IMG_LENGTH": 300,
         "IMG_WIDTH": 300,
         "CHANNELS": 3,
         "RANDOM_STATE": 100,
      "BATCH_SIZE": 50
     }

In [ ]:
Q_range_low = 16
q_range_high = 32
f_range_ll = 10
f_range_high = 400
r1, r2, r3 = read_file("../input/g2net-gravitational-wave-detection/train/0/0/0/00000e74ad.npy")
p1, p2, p3 = preprocess(r1, r2, r3)
hq1 = p1.q_transform(qrange=(Q_range_low, q_range_high), frange=( f_range_ll, f_range_high), logf=True, whiten=False)
hq2 = p2.q_transform(qrange=(Q_range_low, q_range_high), frange=( f_range_ll, f_range_high), logf=True, whiten=False)
hq3 = p3.q_transform(qrange=(Q_range_low, q_range_high), frange=( f_range_ll, f_range_high), logf=True, whiten=False)
#img = np.zeros([hq1.shape[0], hq1.shape[1], 3], dtype=np.float32)
scaler = MinMaxScaler()
hq1 = scaler.fit_transform(hq1)
hq2 = scaler.fit_transform(hq2)
hq3=  scaler.fit_transform(hq3)
img = np.dstack( ( hq1, hq2, hq3 ) )

In [ ]:

def decode_numpy_file ( ):
    def convert_data_3channel_qtran( file_name, target   ):
        
        return  tf.io.decode_jpeg( tf.io.read_file( file_name)),target 
    
    return convert_data_3channel_qtran


def data_augment ():
    
    def perform_aument(img, target ):
        img = tf.image.ran
        
    
    return perform_aument


def datagenerator(df, test = False  ):
    
    read_reshape_gw = decode_numpy_file()
    #augment =  # augmentation Function still in development 
    datagen = tf.data.Dataset.from_tensor_slices( ( df["path_new"].values, df["target"].values ) )
    datagen = datagen.map( read_reshape_gw, num_parallel_calls= tf.data.AUTOTUNE )
    #datagen = datagen.map( read_reshape_gw, num_parallel_calls= tf.data.AUTOTUNE )
    datagen = datagen.repeat() if not test else datagen
    datagen = datagen.shuffle(1024) if not test else datagen
    datagen = datagen.batch(CFG["BATCH_SIZE"])
    datagen = datagen.prefetch(tf.data.AUTOTUNE )
    
    return datagen

In [ ]:
master_df["path_new"]= master_df["id"].parallel_apply( lambda x : "../input/g2net-train-images-with-gpwy-sample/kaggle/tmp/train/" + x+".png")

In [ ]:
png_file_list = [ x.split("/")[-1].replace(".png","") for x in glob.glob("../input/g2net-train-images-with-gpwy-sample/kaggle/tmp/train/*.png")]
master_df_2 = pd.merge( master_df, pd.DataFrame({"png_file": png_file_list,"status":[1]* len( png_file_list) } ), left_on= "id",right_on="png_file" )

In [ ]:
master_df_2["path_new"] = master_df_2["id"].parallel_apply( lambda x : "../input/g2net-train-images-with-gpwy-sample/kaggle/tmp/train/" + x+".png")
train_df, val_df = train_test_split( master_df_2, test_size =0.1, random_state = CFG["RANDOM_STATE"],shuffle = True,stratify = master_df_2["target"])

In [ ]:
fig,axis = plt.subplots( nrows = 1, ncols = 2,figsize =( 15,5), sharey = True ) 
sns.countplot(  val_df["target"], ax = axis[0])
sns.countplot( train_df["target"], ax = axis[1])
axis[0].set_title("Number of Validation Sample with distribution ")
axis[1].set_title("Number of Test Sample with distribution ")
plt.tight_layout( )
plt.show()

In [ ]:
def short_effnet_model():
    #with strategy.scope():
        
    model_input = tf.keras.layers.Input( shape=  ( CFG["IMG_LENGTH"],CFG["IMG_WIDTH"], 3 ) , name= "encoder_input_layer" )


    efff_net =efn.EfficientNetB0(include_top = False, 
                                   weights ="noisy-student" , 
                                   input_shape = ( CFG["IMG_LENGTH"], CFG["IMG_WIDTH"], CFG["CHANNELS"]) ,
                                   input_tensor = model_input ,
                                   classes=2,
                                   pooling = True,
                                   #classifier_activation='softmax',
                                   drop_connect_rate= 0.7
                                  ) 

    for layer in  efff_net.layers  : layer.trainable = True
    
    gaussian_noise = tf.keras.layers.GaussianNoise( stddev = 0.3 ) ( model_input )
    random_crop = tf.keras.layers.experimental.preprocessing.RandomCrop( height = 30, width = 30  ) (gaussian_noise)
    random_flip =tf.keras.layers.experimental.preprocessing.RandomFlip( mode="horizontal_and_vertical") ( random_crop )
    zoom_layer = tf.keras.layers.experimental.preprocessing.RandomZoom(  height_factor =(-0.3, -0.2)  , width_factor=(-0.3, -0.2), fill_mode='reflect', interpolation='bilinear', fill_value=0.0 ) ( random_flip)
    random_contrast = tf.keras.layers.experimental.preprocessing.RandomContrast( factor =[0.2, 0.8 ]  ) ( zoom_layer )
    
    efff_net.layers[0] ( random_contrast )
    layer_00 = efff_net.layers[-1].output
    layer_01 = tf.keras.layers.Flatten()( layer_00 )
    layer_02 = tf.keras.layers.Dense( 1, activation ="sigmoid") ( layer_01)
    model_short = tf.keras.Model( inputs = model_input, outputs = layer_02 )

    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.00126000004/4 ) 
    model_short.compile( optimizer= optimizer,loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                 metrics=[tf.keras.metrics.AUC() ])#AUC(curve='ROC')
    
    return model_short

In [ ]:
model_1 = short_effnet_model()
model_1.summary()

In [ ]:
train_df.shape[0]/10000

In [ ]:
if False:
    
    CFG["BATCH_SIZE"]  = 50
    train_df_2 = train_df.head(50000)
    val_df_2 = val_df.head( 50000 )
    train_data_gen = datagenerator( train_df_2 )
    val_data_gen = datagenerator( val_df_2 )

    model_effnet = short_effnet_model()
    lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(  patience=2,
                                                        min_lr= 0.000001,
                                                        monitor='val_loss', 
                                                        factor=0.45, 
                                                        verbose=1,
                                                        min_delta = 0.2,
                                                        cooldown=2,
                                                        mode='auto', 
                                                       )







    CFG["TRAIN_STEPS"] = int ( train_df_2.shape[0] /CFG["BATCH_SIZE"] ) + (1 if train_df_2.shape[0] % CFG["BATCH_SIZE"] != 0 else 0)
    CFG["VAL_STEPS"] = int ( val_df_2.shape[0]/CFG["BATCH_SIZE"] ) + (1 if val_df_2.shape[0]% CFG["BATCH_SIZE"] != 0 else 0)



    #model_effnet.load_weights("../input/seti-gpu-rev-01-model/Efficient_Net_Model_Rev_01.h5")
    checkpoint = tf.keras.callbacks.ModelCheckpoint( f'model{1}.h5', save_best_only=True, monitor='val_loss', mode='min')


    model_history = model_effnet.fit( train_data_gen ,
                            #class_weight= class_weight ,
                             steps_per_epoch= CFG["TRAIN_STEPS"], 
                             epochs =11, 
                             validation_data= val_data_gen,
                             validation_steps = CFG["VAL_STEPS"],
                             callbacks=[ checkpoint,lr_reducer ]
                           )


# Code for TF record 

In [ ]:
CREATE_TF_RECORD = False

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    #if isinstance(value, type(tf.constant(0))):
    #    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def train_serialize_example(image, img_id, target ):
    feature = {
      'image'         : _bytes_feature(image),
      'image_id'      : _bytes_feature(img_id),   
      'target'        : _int64_feature(target),
      }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()


if CREATE_TF_RECORD :
    
    ! mkdir "./300x300_tf_record"
    each_tfrec_size = 10000
    
    if (master_df_2.shape[0] % each_tfrec_size) != 0 :
        end  = 1 + int(master_df_2.shape[0] / each_tfrec_size)
    else:
        end = int( master_df_2.shape[0] / each_tfrec_size )
        
        
    for i in range( 0, end):
        
        
        if ((i +1)* each_tfrec_size) > master_df_2.shape[0]:
            
            img_id = master_df_2["id"].iloc[ i*each_tfrec_size : ].values
            file_name = master_df_2["path_new"].iloc[ i*each_tfrec_size : ].values
            target = master_df_2["target"].iloc[ i*each_tfrec_size : ].values
            
        else:
            
            img_id= master_df_2["id"].iloc[ i*each_tfrec_size : each_tfrec_size *(i +1)].values
            file_name = master_df_2["path_new"].iloc[ i*each_tfrec_size : each_tfrec_size *(i +1)].values
            target = master_df_2["target"].iloc[ i*each_tfrec_size : each_tfrec_size *(i +1)].values
        
        print ( "Ongoing slice {}/{}".format( i,end))
        
        with tf.io.TFRecordWriter( "./300x300_tf_record/300x300_tfrecord_" +str( i) + ".tfrec" ) as writer:
            
                    for each_id, each_file, each_target  in zip( img_id,file_name, target ):
                        image_string = open(each_file, 'rb').read()
                        
                        writer.write(train_serialize_example( image_string , str.encode(each_id ), each_target ))
                        
                    writer.close()
                    
        print ( "completed slice {}/{}".format( i,end))
    

In [ ]:
import os
os.chdir(r'kaggle/working')

In [ ]:
# code decode tfrecode 
def decode_image(image_data):
    image = tf.io.decode_png( image_data,tf.float32 )
    return image

def prepare_target(target):    
    target = tf.cast(target, tf.float32)            
    target = tf.reshape(target, [1])         
    return target

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image" : tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "image_id":tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = tf.io.decode_png( example['image'])/ 255
    #image  = tf.reshape(image, [256, 256])
    #image = tf.stack( (image, image, image), axis = 2)
    target = prepare_target(example['target'])
    return image, target # returns a dataset of (image, label) pairs

def augmanet_data(image, target ):
    
    #mask = random.randrange(2, 40, 2)
   
    #offset = random.randrange( 1, 200, 2 )
    
    #image =  tf.image.random_contrast( tf.image.random_flip_up_down( tf.image.random_flip_left_right( image, seed=CFG["RANDOM_STATE"]  ),  seed=CFG["RANDOM_STATE"] 
    #                                                           ),0.3,0.8, seed=CFG["RANDOM_STATE"] )
    
    #image= tf.squeeze(tfa.image.random_cutout( tf.expand_dims(image,0), (10, 10) ) )
    #image = tfa.image.cutout( tf.expand_dims(image,0),(10,10), constant_values = 0.0,offset = (2,2,2) )
    #image = tfa.image.cutout( images= image, mask_size = (mask,mask), constant_values = 0  )#, offset =(2,2 ), constant_values = 0)
      
    return image , target
    
                                        

def load_dataset(fileids, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(fileids, num_parallel_reads=tf.data.AUTOTUNE) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord,num_parallel_calls= tf.data.AUTOTUNE)
   # dataset = dataset.map( augmanet_data ,num_parallel_calls= tf.data.AUTOTUNE) if augment else dataset
    
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset


## Main function 
def get_training_dataset(file_ist,repeat = True ,ordered =False  ):
    dataset = load_dataset(file_ist, ordered = False )
    dataset = dataset.repeat()  if repeat else  dataset # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(20, seed=CFG["RANDOM_STATE"])
    dataset = dataset.batch(CFG["BATCH_SIZE"])
    dataset = dataset.prefetch(tf.data.AUTOTUNE) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset


data_gen = get_training_dataset( ["./300x300_tf_record/300x300_tfrecord_0.tfrec"],repeat = False, ordered = True  )


In [ ]:
import os
os.chdir(r'/kaggle/working') 

In [ ]:
os.getcwd()

In [ ]:
%cd /kaggle/working

In [ ]:
! mkdir "300x300_tf_record/"

In [ ]:
import shutil
for i in glob.glob("./300x300_tf_record/*.tfrec"):
    shutil.move(i, "/kaggle/working/")

In [ ]:
ls


In [ ]:
file_list = """300x300_tfrecord_18.tfrec
300x300_tfrecord_28.tfrec
300x300_tfrecord_0.tfrec
300x300_tfrecord_19.tfrec
300x300_tfrecord_3.tfrec
300x300_tfrecord_1.tfrec
300x300_tfrecord_2.tfrec
300x300_tfrecord_4.tfrec
300x300_tfrecord_10.tfrec
300x300_tfrecord_20.tfrec
300x300_tfrecord_5.tfrec
300x300_tfrecord_11.tfrec
300x300_tfrecord_21.tfrec
300x300_tfrecord_6.tfrec
300x300_tfrecord_12.tfrec
300x300_tfrecord_22.tfrec
300x300_tfrecord_7.tfrec
300x300_tfrecord_13.tfrec
300x300_tfrecord_23.tfrec
300x300_tfrecord_8.tfrec
300x300_tfrecord_14.tfrec
300x300_tfrecord_24.tfrec
300x300_tfrecord_9.tfrec
300x300_tfrecord_15.tfrec
300x300_tfrecord_25.tfrec
300x300_tfrecord_16.tfrec
300x300_tfrecord_26.tfrec 
300x300_tfrecord_17.tfrec
300x300_tfrecord_27.tfrec"""

In [ ]:
new_file_list =  [ x for x in file_list.replace(" ","").split("\n") if x !="" ] 

In [ ]:
import time 

In [ ]:
from IPython.display import publish_display_data

In [ ]:
#publish_display_data( {"a":new_file_list[0] } )

In [ ]:
#FileLink(r"300x300_tfrecord_17.tfrec")

In [ ]:
help( publish_display_data 
    )

In [ ]:
from IPython.display import FileLink
    for i in new_file_list: 
        FileLink( i )


In [ ]:
#FileLink( new_file_list[9] )